In [ ]:
# Import Packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import svm
from sklearn import tree
from sklearn.feature_selection import SequentialFeatureSelector as sfs
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")


# Load Dataset
wine_data = pd.read_csv('data.csv')
wine_data.drop(['Id'], axis=1, inplace=True)
wine_data.head()

In [ ]:
wine_data.info()

In [ ]:
target_variable = 'quality'
features = [col for col in wine_data.columns if col != target_variable]
wine_data[features].describe()

In [ ]:
def descriptive_categorical(dataframe, col_name, cmap=None, alpha_sort=False):
    if cmap is None:
        cmap='gist_earth_r'
    data = pd.DataFrame(dataframe[col_name].value_counts(normalize=True).sort_values(ascending=False)*100).reset_index()
    data.rename(columns={'index': col_name, col_name: '% observations'}, inplace=True)
    if alpha_sort:
        data.sort_values(by=col_name, inplace=True)
    data['Cumulative %'] = data['% observations'].cumsum()
    data_styled = data.style.background_gradient(cmap=cmap, subset=['% observations','Cumulative %'])\
        .format(formatter={'% observations': "{:.2f}%", 'Cumulative %':"{:.2f}%"}).hide_index()
    return data_styled

descriptive_categorical(dataframe=wine_data, col_name='quality', cmap='Greys', alpha_sort=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
sns.histplot(data=wine_data, x='quality', stat='count', color='slategray')
ax.grid(False)

In [ ]:
wine_data['good_quality'] = np.where(wine_data['quality'] >= 6, 1, 0)
fig, ax = plt.subplots(figsize=(8, 4))
sns.histplot(data=wine_data, x='quality', hue='good_quality', stat='count', palette=['#929292', '#2ca02c'])
ax.set_title('Wine Quality Score Distribution')
ax.grid(False)

In [ ]:
descriptive_categorical(dataframe=wine_data, col_name='good_quality', cmap='Greens', alpha_sort=True)

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(12, 9), layout='constrained')
for ax, field in zip(axs.flat, features):
    if field is not None:
        sns.histplot(data=wine_data, x=field, hue='good_quality', kde=True, palette=['#929292', '#2ca02c'], ax=ax)
        ax.legend(['good wine', 'bad wine'])
        ax.set_title(f'\n{field} variable')
        ax.grid(False)

In [ ]:
model_data = pd.DataFrame(wine_data)
model_data.drop(labels='quality', axis=1, inplace=True)
model_data.head()

In [ ]:
target_variable = 'good_quality'
full_model_data = pd.DataFrame(model_data)
full_model_features = full_model_data.loc[:, full_model_data.columns != target_variable].columns
full_X = sm.add_constant(full_model_data[full_model_features])
full_y = full_model_data[target_variable]
full_model = sm.GLM(full_y, full_X, family=sm.families.Binomial()).fit()
print(full_model.summary())

In [ ]:
def get_outliers(model, conditions=None):
    """
    model: statsmodels.api model
    returns: dataframe of outliers based on cook's distance threshold (4/n) and >= 3 standard deviations 
    of the standardized residuals.
    """
    
    summary_df = model.get_influence().summary_frame()
    results_df = summary_df[['cooks_d']]
    results_df['cooks_threshold'] = (4 / model.nobs)
    results_df['std_residual'] = stats.zscore(model.resid_pearson)
    results_df['std_residual'] = np.abs(results_df['std_residual'])
    results_df['std_threshold'] = 3
    choices = ['outlier']
    conditions = [(results_df['cooks_d'] > (4 / model.nobs)) & (results_df['std_residual'] > 3)]
    results_df['outlier'] = np.select(conditions, choices, default='not outlier')
    return results_df

distances = get_outliers(full_model)
outliers = pd.DataFrame(distances[distances['outlier']=='outlier'])
outliers.sort_values('cooks_d', ascending=False, inplace=True)
outliers.head()

In [ ]:
results_df = get_outliers(full_model)
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(12, 4))
sns.histplot(data=results_df, x='std_residual', hue='outlier', palette=['#1E88E5', '#D81B60'], bins='auto', ax=ax[0], kde=True)
ax[0].set_xlabel('Residual Standard Deviation')
ax[0].grid(False)
sns.histplot(data=results_df, x='cooks_d', hue='outlier', palette=['#1E88E5', '#D81B60'], bins='auto', ax=ax[1], kde=True)
ax[1].set_xlabel('Residual Cooks Distance')
ax[1].grid(False)
fig.suptitle('Full Model Residuals')
plt.show()

In [ ]:
data_no_outliers = pd.DataFrame(model_data)
data_no_outliers.drop(labels=outliers.index, axis=0, inplace=True)
target_variable = 'good_quality'
full_model_features = data_no_outliers.loc[:, data_no_outliers.columns != target_variable].columns
full_X = sm.add_constant(data_no_outliers[full_model_features])
full_y = data_no_outliers[target_variable]
full_model_no_outliers = sm.GLM(full_y, full_X, family=sm.families.Binomial()).fit()
print(full_model_no_outliers.summary())

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(12, 9), layout='tight')
for ax, field in zip(axs.flat, features):
    if field is not None:
        new_df = pd.DataFrame(wine_data)
        X = new_df[field]
        y = new_df[target_variable]
        logit_results = sm.GLM(y, X, family=sm.families.Binomial()).fit()
        predicted = logit_results.predict(X)
        log_odds = np.log(predicted / (1 - predicted))
        a, b = np.polyfit(wine_data[field], log_odds, 1)
        ax.scatter(x=wine_data[field], y=log_odds, color='cornflowerblue', label='observations')
        ax.plot(wine_data[field], a*wine_data[field]+b, color='maroon', label='line of best fit')     
        ax.set_title(f'\n{field} variable')
        ax.set_ylabel('Log Odds')
        ax.legend()
        ax.grid(False)
plt.show()

In [ ]:
def get_model_vif(X_variables_as_df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_variables_as_df.columns
    vif_data["VIF"] = [variance_inflation_factor(X_variables_as_df.values, i) for i in range(len(X_variables_as_df.columns))]
    return vif_data.sort_values(by='VIF', ascending=False)


full_model_vif = get_model_vif(full_model_data[full_model_features])
full_model_vif

In [ ]:
def correlation_matrix(X_variables_as_df, figsize=(10,8), cmap='Blues', annot=True):
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(X_variables_as_df.corr(), cmap=cmap, ax=ax, annot=annot)
    ax.set_title('Correlation Matrix')
    plt.show()
    
correlation_matrix(full_model_data[full_model_features], figsize=(10,8), cmap='Blues', annot=True)

In [ ]:
print(full_model.summary())

In [ ]:
model = linear_model.LogisticRegression(fit_intercept=True)
X_train, X_test, y_train, y_test = train_test_split(full_model_data[full_model_features], full_model_data[target_variable],
                                                    test_size=0.2, random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)
score = np.round(accuracy_score(y_test, pred), 2)
cm = confusion_matrix(y_test, model.predict(X_test))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['good quality', 'bad quality'], yticklabels=['good quality', 'bad quality'], cmap=plt.cm.get_cmap('cividis').reversed())
plt.title('Full Model Logistic Regression' + ' Accuracy: ' + str(score*100) + '%')
plt.xlabel('Predicted Outcome')
plt.ylabel('Actual Outcome')
plt.show()

In [ ]:
results_df = pd.DataFrame(columns=['Model Name', 'Model Type', 'Mean Training Accuracy', 'Mean Validation Accuracy'])

def plot_k_fold_validation(algo, results_df, model_name, model_type, X, y, palette=None, folds=5):
    if palette is None:
        palette= ['#1f77b4', '#aec7e8']
    plot_title = model_name + ' ' + model_type + ': ' + str(folds) + '-Fold Validation'
    validation = cross_validate(estimator=algo.fit(X, y), X=X, y=y, cv=folds, scoring=['accuracy', 'precision', 'recall', 'f1'], return_train_score=True)
    train_accuracy = np.round(np.mean(validation['train_accuracy'])*100,2)
    test_accuracy = np.round(np.mean(validation['test_accuracy'])*100,2)
    print('Mean Training Accuracy score: ', str(train_accuracy))
    print('Mean Testing Accuracy score: ', str(test_accuracy))
    fig, ax = plt.subplots(figsize=(10, 6))
    labels = labels = ['Fold ' + str(i+1) for i in range(len(validation['train_accuracy']))]
    x = np.arange(len(labels))
    ax.set_ylim(0.25, 1)
    for i, j in enumerate(validation['train_accuracy']):
        ax.text(x[i] - 0.3, j + 0.02, str(np.round(j, 2)), color=palette[0], fontweight='bold')
    for i, j in enumerate(validation['test_accuracy']):
        ax.text(x[i] + 0.1, j + 0.02, str(np.round(j, 2)), color=palette[1], fontweight='bold')
    ax.bar(x - 0.2, validation['train_accuracy'], 0.4, color=palette[0], label='Training')
    ax.bar(x + 0.2, validation['test_accuracy'], 0.4, color=palette[1], label='Testing')
    ax.set_title(plot_title, fontsize=16)
    ax.set_xticks(x, labels)
    ax.set_xlabel(model_name + ' ' + model_type, fontsize=14)
    ax.set_ylabel('Accuracy', fontsize=14)
    ax.legend()
    ax.grid(False)
    plt.show()
    results_df.loc[len(results_df.index)] = [model_type, model_name, train_accuracy, test_accuracy]
    
    
plot_k_fold_validation(linear_model.LogisticRegression(fit_intercept=True), results_df, 'Full Model', 
                       'Logistic Regression', full_model_data[full_model_features], full_model_data[target_variable], 
                       palette=['#d62728', '#ff9896'], folds=5)

In [ ]:
full_model_vif

In [ ]:
reduced_model_features = list(full_model_features)
reduced_model_features.remove('density')
reduced_model_features.remove('pH')
reduced_model_vif = get_model_vif(full_model_data[reduced_model_features])
reduced_model_vif.head(10)

In [ ]:
correlation_matrix(full_model_data[reduced_model_features])

In [ ]:
target_variable = 'good_quality'
model_1_data = pd.DataFrame(model_data)
model_1_features = list(reduced_model_features)
model_1X = sm.add_constant(model_1_data[model_1_features])
model_1y = model_1_data[target_variable]
model_1 = sm.GLM(model_1y, model_1X, family=sm.families.Binomial()).fit()
print(model_1.summary())

In [ ]:
plot_k_fold_validation(linear_model.LogisticRegression(fit_intercept=True), results_df, 'Reduced Model 1', 
                       'Logistic Regression', model_1_data[model_1_features], model_1_data[target_variable], 
                       palette=['#ff7f0e', '#ffbb78'], folds=5)

In [ ]:
tols = [0.1, 0.00001, 0.000000000001, 0.0000000000000000000001, 0.0000000000000000000000000000000000000000001
       ,0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000001]
model_2_features = None
for i, tol in enumerate(tols):
    selector = sfs(linear_model.LogisticRegression(fit_intercept=True), n_features_to_select='auto', tol=tol, scoring='accuracy')
    results = selector.fit(full_model_data[full_model_features], full_model_data['good_quality'])
    if i !=0:
        model_features = selector.get_feature_names_out()
        if model_features.all() != model_2_features.all():
            model_2_features = model_features
    model_2_features = selector.get_feature_names_out()
print('Reduced Features: ', tol, model_2_features)

In [ ]:
model_2_data = pd.DataFrame(model_data)
target_variable = 'good_quality'
model_2X = sm.add_constant(model_2_data[model_2_features])
model_2y = model_2_data[target_variable]
model_2 = sm.GLM(model_2y, model_2X, family=sm.families.Binomial()).fit()
print(model_2.summary())

In [ ]:
plot_k_fold_validation(linear_model.LogisticRegression(fit_intercept=True), results_df, 'Reduced Model 2', 
                       'Logistic Regression', model_2_data[model_2_features], model_2_data[target_variable], 
                       palette=['#bcbd22', '#dbdb8d'], folds=5)

In [ ]:
print(model_1.summary())

In [ ]:
model_log_data = pd.DataFrame(model_data)
for feature in model_1_features:
    model_log_data[f'{feature}:log_{feature}'] = model_log_data[feature].apply(lambda x: x * np.log(x))
model_log_data.fillna(value=0, axis=1, inplace=True)

log_X = sm.add_constant(model_log_data)
for col in log_X.columns:
    if ('log' not in col or col == target_variable) or ('pH' in col or 'density' in col):
        log_X.drop(labels=col, axis=1, inplace=True)
log_X = sm.add_constant(log_X)
log_model = sm.GLM(model_log_data[target_variable], log_X, family=sm.families.Binomial()).fit()
print(log_model.summary())

In [ ]:
model_sq_data = pd.DataFrame(model_data)
for feature in full_model_features:
    model_sq_data[f'{feature}:sq_{feature}'] = model_sq_data[feature].apply(lambda x: x**2)
model_sq_data.fillna(value=0, axis=1, inplace=True)

sq_X = sm.add_constant(model_sq_data)
for col in sq_X.columns:
    if ('sq' not in col or col == target_variable) or ('pH' in col or 'density' in col):
        sq_X.drop(labels=col, axis=1, inplace=True)
sq_X = sm.add_constant(sq_X)
sq_model = sm.GLM(model_sq_data[target_variable], sq_X, family=sm.families.Binomial()).fit()
print(sq_model.summary())

In [ ]:
model_reciprocal_data = pd.DataFrame(full_model_data)
for feature in full_model_features:
    model_reciprocal_data[f'{feature}:recip_{feature}'] = np.divide(1,  model_reciprocal_data[feature], where= model_reciprocal_data[feature]!=0)
model_reciprocal_data.fillna(value=0, axis=1, inplace=True)

recip_X = sm.add_constant(model_reciprocal_data)
for col in recip_X.columns:
    if ('recip' not in col or col == target_variable) or ('pH' in col or 'density' in col):
        recip_X.drop(labels=col, axis=1, inplace=True)
recip_X = sm.add_constant(recip_X)

reciprocal_model = sm.GLM(model_reciprocal_data[target_variable], recip_X, family=sm.families.Binomial()).fit()
print(reciprocal_model.summary())

In [ ]:
model_3_data = pd.DataFrame(model_data)
recip_columns = pd.DataFrame(model_reciprocal_data[['chlorides:recip_chlorides', 'free sulfur dioxide:recip_free sulfur dioxide', 'residual sugar:recip_residual sugar']])
model_3_features = list(model_1_features)
model_3_features.remove('residual sugar')
model_3_features.remove('free sulfur dioxide')
model_3_features.remove('chlorides')
model_3X = pd.concat([model_3_data[model_3_features], recip_columns], axis=1)
model_3X = sm.add_constant(model_3X)
model_3y = model_3_data[target_variable]
model_3 = sm.GLM(model_3y, model_3X, family=sm.families.Binomial()).fit()
print(model_3.summary())

In [ ]:
plot_k_fold_validation(linear_model.LogisticRegression(fit_intercept=True), results_df, 'Reduced Model 3', 
                       'Logistic Regression', model_3_data[model_3_features], model_3_data[target_variable], 
                       palette=['#2ca02c', '#98df8a'], folds=5)
plot_k_fold_validation(linear_model.LogisticRegression(fit_intercept=True), results_df, 'Reduced Model 2', 
                       'Logistic Regression', model_2_data[model_2_features], model_2_data[target_variable], 
                       palette=['#bcbd22', '#dbdb8d'], folds=5)
plot_k_fold_validation(linear_model.LogisticRegression(fit_intercept=True), results_df, 'Reduced Model 1', 
                       'Logistic Regression', model_1_data[model_1_features], model_1_data[target_variable], 
                       palette=['#ff7f0e', '#ffbb78'], folds=5)

In [ ]:
plot_k_fold_validation(linear_model.RidgeClassifierCV(fit_intercept=True), results_df, 'Reduced Model 3', 
                       'Ridge Classifier', model_3_data[model_3_features], model_3_data[target_variable], 
                       palette=['#17becf', '#9edae5'], folds=5)

In [ ]:
plot_k_fold_validation(svm.LinearSVC(fit_intercept=True, random_state=24), results_df, 'Reduced Model 3', 
                       'SVM Classifier', model_3_data[model_3_features], model_3_data[target_variable], 
                       palette=['#1f77b4', '#aec7e8'], folds=5)

In [ ]:
plot_k_fold_validation(neighbors.KNeighborsClassifier(), results_df, 'Reduced Model 3', 
                       'KNN Classifier', model_3_data[model_3_features], model_3_data[target_variable], 
                       palette=['#9467bd', '#c5b0d5'], folds=5)

In [ ]:
plot_k_fold_validation(naive_bayes.GaussianNB(), results_df, 'Reduced Model 3', 
                       'Naive Bayes (Gaussian NB)', model_3_data[model_3_features], model_3_data[target_variable], 
                       palette=['#8c564b', '#c49c94'], folds=5)

In [ ]:
depths = [1, 2, 3, 5, 10, 25]
for depth in depths:
    model_type = 'Decision Tree Classifier (Max Depth = '+str(depth)+')'
    model_name = 'Reduced Model 3'
    plot_k_fold_validation(tree.DecisionTreeClassifier(criterion='log_loss', max_depth=depth), results_df, model_name, 
                           model_type, model_3_data[model_3_features], model_3_data[target_variable], 
                           palette=['#7f7f7f', '#c7c7c7'], folds=5)

In [ ]:
results_df.drop_duplicates(inplace=True)
results_df.sort_values(by='Mean Validation Accuracy', ascending=False, inplace=True)
results_df.style.bar(cmap='Spectral').format(formatter={'Mean Training Accuracy': "{:.2f}%", 'Mean Validation Accuracy':"{:.2f}%"})\
        .hide_index()